In [5]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import requests
import torch.nn.functional as F
import os


model = models.resnet18(weights="ResNet18_Weights.DEFAULT")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

torch.save(model.state_dict(), 'models/resnet18.pth')
model = models.resnet18()
model.load_state_dict(torch.load('models/resnet18.pth'))
model.to(device)

model.eval()

# load the ImageNet class labels
labels_url = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
labels = requests.get(labels_url).json()

def preprocess_image(image):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    preprocessed_image = preprocess(image).unsqueeze(0).to(device)
    return preprocessed_image  # Add batch dimension


def classify_image(image_tensor):
    # Perform inference
    with torch.no_grad():
        outputs = model(image_tensor)

    # Get the predicted class
    _, predicted = torch.max(outputs, 1)

    # Print the predicted class label
    print("Predicted class:", predicted.item())


image_path = "LPIPS_Images/Lpips_image_1.jpg" 
image = Image.open(image_path)
image_tensor = preprocess_image(image)

classify_image(image_tensor)

Predicted class: 51


In [4]:
# making ground label
def ground_truth(true_class):
    truth_tensor = [0]*1000
    class_index = labels.index(true_class)
    # print("class index of "+ true_class, class_index)
    truth_tensor[class_index] = 1
    truth_tensor = torch.tensor(truth_tensor).unsqueeze(0).to(device)
    truth_tensor = truth_tensor.to(torch.float32)

    return truth_tensor

directory = "LPIPS_Images"
images_path = os.listdir(directory)

for file in images_path:
    file = os.path.join(directory, file)
    image = Image.open(file)
    image_tensor = preprocess_image(image)

    # Perform inference
    with torch.no_grad():
        outputs = model(image_tensor)

    # compute the loss
    ground_truth_tensor = ground_truth('triceratops')
    loss = F.cross_entropy(outputs,ground_truth_tensor)
    print("Prediction loss:", loss.item(), " for ", file)



Prediction loss: 0.19428707659244537  for  LPIPS_Images\Lpips_image_0.jpg
Prediction loss: 0.19406099617481232  for  LPIPS_Images\Lpips_image_1.jpg
Prediction loss: 0.36393511295318604  for  LPIPS_Images\Lpips_image_2.jpg
Prediction loss: 0.5075499415397644  for  LPIPS_Images\Lpips_image_3.jpg
Prediction loss: 2.0331428050994873  for  LPIPS_Images\Lpips_image_4.jpg
Prediction loss: 3.7291922569274902  for  LPIPS_Images\Lpips_image_5.jpg
Prediction loss: 4.2316365242004395  for  LPIPS_Images\Lpips_image_6.jpg
Prediction loss: 5.045066833496094  for  LPIPS_Images\Lpips_image_7.jpg
Prediction loss: 5.67869234085083  for  LPIPS_Images\Lpips_image_8.jpg
Prediction loss: 5.892947673797607  for  LPIPS_Images\Lpips_image_9.jpg
Prediction loss: 11.069872856140137  for  LPIPS_Images\sit.jpg
